# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [8]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler    
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from loguru import logger

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(20, 20)

sns.set()

In [3]:
fifa = pd.read_csv("fifa.csv")

In [4]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [5]:
# Sua análise começa aqui.
#Avaliando total de linhas e colunas
fifa.shape

(18207, 37)

In [17]:
#Conhecendo as colunas
fifa.columns

Index(['Age', 'Overall', 'Potential', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes'],
      dtype='object')

In [18]:
#removendo valores nulos
fifa = fifa.dropna()

In [19]:
#separação entre feature e target
x = fifa.drop(columns='Overall').values
y = fifa.Overall
x = StandardScaler().fit_transform(x)

In [20]:
#Questão 01 - fração da variância para a PC1
pca = PCA().fit(x)
evr = pca.explained_variance_ratio_
evr.round(3)

array([0.526, 0.137, 0.083, 0.049, 0.037, 0.033, 0.017, 0.012, 0.01 ,
       0.008, 0.008, 0.007, 0.006, 0.006, 0.006, 0.006, 0.005, 0.005,
       0.004, 0.004, 0.003, 0.003, 0.003, 0.003, 0.002, 0.002, 0.002,
       0.002, 0.002, 0.002, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

In [23]:
var_acumm = np.cumsum(evr)
type((np.argmax(var_acumm >= 0.95) + 1).item())

int

In [24]:
#questão 03
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]
pca.components_

array([[-0.02400556, -0.069642  , -0.19982198, ...,  0.19376951,
         0.19378084,  0.1946677 ],
       [ 0.06952447, -0.02858912, -0.04083772, ..., -0.07908306,
        -0.07694267, -0.07824029],
       [ 0.36348672,  0.20828337,  0.0219397 , ...,  0.24114171,
         0.24622027,  0.24229328],
       ...,
       [ 0.00773915,  0.00086435, -0.01314651, ..., -0.04569521,
        -0.57774798, -0.10774643],
       [ 0.00087493,  0.00140355, -0.00192555, ..., -0.0338206 ,
        -0.03029814,  0.72265415],
       [ 0.00258467, -0.00391813,  0.00884775, ...,  0.00807702,
         0.03824106,  0.04395471]])

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [28]:
def q1():
    pca = PCA().fit(fifa)
    evr = pca.explained_variance_ratio_
    return round(np.cumsum(evr)[0].item(),3)

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [182]:
def q2():
    pca = PCA().fit(fifa)
    evr = pca.explained_variance_ratio_
    var_acumm = np.cumsum(evr)
    return (np.argmax(var_acumm >= 0.95) + 1).item()

15

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [8]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [161]:
def q3():
    pca = PCA().fit(fifa)
    pc = tuple(pca.components_.dot(x).round(3))
    return pc[0:2]
q3()

(186.556, -6.592)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [16]:
def q4():
    x = fifa.drop(columns='Overall')
    y = fifa.Overall
    linear = LinearRegression()
    rfe = RFE(estimator=linear, n_features_to_select=5).fit(x, y)
    return list(x.columns[rfe.support_])